Jugando con [CID](https://docs.ipfs.eth.link/concepts/content-addressing/).

CID, se define como una etiqueta autodescriptiva, que identifica el contenido, que se utiliza para señalar los datos almacenados en IPFS. Este ejemplo pretende aclarar esta definición.

Existen 2 versiones para representar un CID, basicamente porque se partió de una versión, que ahora se conoce como v0 que fue mejorada a v1 para soportar más formatos y ser más flexible:


CID v0:

* Siempre comienza con `Qm`.
* Solo admite un formato fijo `dag-pb` y hash SHA-256, sin soporte para otros codecs o hashes.
  > `dag-pb` es el formato de serialización por defecto en CID v0, basado en [Protocol Buffers](https://protobuf.dev/), utilizado por IPFS para contener el bloque del contenido, que incluye metadatos UnixFS del archivos o directorios que forma parte y las relaciones a bloques hijos. Ve a la práctica para [entender IPLD](practice-understand-IPLD.md) si quieres más contexto.
* Sensible a mayúsculas/minúsculas, menos adecuado para URLs.

CID v1:

* Puede comenzar con distintos prefijos (b, z, etc.) según la codificación (Base32, Base58...).
* Soporta múltiples codecs para identificar mejor el fragmento del contenido y puede usar varios algoritmos de hash.
* No distingue mayúsculas/minúsculas, ideal para usar en URLs y subdominios.

---

**Ejemplo v1**.

El ejemplo que se muestra se basa en CID Inspector, por lo que debes acceder a:
<https://cid.ipfs.tech/#bafkreifzlpgwving46m34a6exyiaj6a6fwruydheyvwvqv36jel73jyjja>


Aquí se intenta aclarar cada concepto paso a paso.

> Por favor, ejecuta una a una cada celda de códgo para que no falle.

In [1]:
cid = "bafkreifzlpgwving46m34a6exyiaj6a6fwruydheyvwvqv36jel73jyjja"

print(f"El CID {cid}")

El CID bafkreifzlpgwving46m34a6exyiaj6a6fwruydheyvwvqv36jel73jyjja


El 1º caracter, la "b", es un multibase, quiere decir que el contenido puede estar codificado en diferentes bases.

Los posibles valores en <https://github.com/multiformats/multibase>

En este ejemplo, la "b" indica que el resto de contenido es base32 y en minúsculas (porque se tiene en cuenta).

Comunmente puede ser:
- 'z': indica base58btc (CIDv0)
- 'b': base32 (CIDv1, minúsculas).
- 'B': base32 (CIDv1, mayúsculas).
- 'f': base16 (CIDv1, hexadecimal).

> Que empieze por 'Q' indica que es CID v0, no es un multibase.

Así que quitamos el multibase, la "b", y nos quedamos con el resto del valor que es la concatenación de version + multicodec + multihash 

Igualmente tiene el adjetivo "multi" porque implica que puede tener multiples valores, es simplemente una forma de llamarlo, es decir, se refiere al code usado y el valor del hash.

In [2]:
cid_body = cid[1:]  # quitar 'b'
print(f"El valor es {cid_body}")

El valor es afkreifzlpgwving46m34a6exyiaj6a6fwruydheyvwvqv36jel73jyjja


Sabemos que es base32 así que tenemos que tenerlo en cuenta para pasarlo a hexadecimal.

Pero antes corregimos el valor para que coincida con el padding (relleno) de base32, agregando "=" hasta completar la longuitud multiple de 8.

In [3]:
missing_padding = (8 - len(cid_body) % 8) % 8
cid_body_padded = cid_body.upper() + "=" * missing_padding
print(f"Corregimos el padding para pasar a base32 {cid_body_padded}")

Corregimos el padding para pasar a base32 AFKREIFZLPGWVING46M34A6EXYIAJ6A6FWRUYDHEYVWVQV36JEL73JYJJA======


Pasamos de base32 a hexadecimal

In [4]:
import base64
cid_bytes = base64.b32decode(cid_body_padded)
cid_hex = cid_bytes.hex()
print(f"En hexadecimal el valor es 0x{cid_hex}")

En hexadecimal el valor es 0x01551220b95bcd6aa1a6e799be03c4be1004f81e2da34c0ce4c56d58577e4917fda70948


Tenemos que extaer version + multicodec + multihash 

Extraemos el valor de versión, que es la posición 1 y 2.

Las posibles versiones lo vemos en: <https://github.com/multiformats/cid?tab=readme-ov-file#versions>

PD: evidentemente es la version 1, porque el multibase "b" ya nos lo indicaba.

In [5]:
cod_version = cid_hex[0:2]
print(f"Versión: 0x{cod_version}")

Versión: 0x01


> Ahora es obvio que es v1, si fuera v0 empezaría por `Qm`, pero hay que entender que se crea el formato para que pueda evolucionar, por ejemplo, podría ser la v2, que actualmente no existe, pero se deja la posibilidad.

Ahora extraemos el multicodec, que es la posición 3 y 4.

Los posibles valores en <https://github.com/multiformats/multicodec>

El multicodec describe el formato del contenido apuntado por el CID, no el contenido en sí. Le dice al nodo de IPFS cómo decodificar el bloque, que en ejemplos comunes puede ser:
- raw: bytes sin estructura.
- dag-pb: como ya explicamos, estructura tipo protobuf, usado en IPFS para representar directorios y archivos en sistemas tipo UnixFS
- dag-cbor: objetos serializados tipo JSON binario.
- dag-json: JSON directo.

In [6]:
# Para mostrar el nombre del multicodec, usamos la libreria multiformats
from multiformats import multicodec

cod_multicodec = cid_hex[2:4]
cod_multicodec_hex_to_dec = int(cod_multicodec,16) # convertir de hexadecimal a entero
codec = multicodec.get(code=cod_multicodec_hex_to_dec)
print(f"Código multicodec: 0x{cod_multicodec}")
print(f"Nombre y descripción: {codec.name}, {codec.description}")

Código multicodec: 0x55
Nombre y descripción: raw, raw binary


Ahora extraemos el multihash: https://github.com/multiformats/multihash

Está compuesto por el código, longuitud y digest (es decir, el valor tras aplicar la función hash)

El código de multihash, posición 5 y 6.

Indica el algoritmo usado para genear el hash.

In [7]:
cod_multihash = cid_hex[4:6]
cod_multihash_int = int(cod_multihash, 16) # pasarlo de hexadecimal a entero
mh = multicodec.get(code=cod_multihash_int)
print(f"Código: 0x{cod_multihash}")
print(f"Nombre: {mh.name}")

Código: 0x12
Nombre: sha2-256


La longuitud del multihash, posición 7 y 8

> En teoría el código 0x12 (sha2-256) ya implica 32 bytes, pero se incluye la longitud explícitamente por diseño de Multihash.

In [9]:
len_multihash = cid_hex[6:8]
print(f"Longuitud 0x{len_multihash} o {int(len_multihash,16)} bits")

Longuitud 0x20 o 32 bits


El valor digest, es decir, el propio hash aplicando la función concreta que se especifica en multicodec.

Posición desde 9 al final.

In [10]:
digestMultiHash_hex = cid_hex[8:]
print(f"Digest: {digestMultiHash_hex.upper()}")

Digest: B95BCD6AA1A6E799BE03C4BE1004F81E2DA34C0CE4C56D58577E4917FDA70948


Ahora el propio multihash, se puede representar en base32 (la propia multibase) para que sea más legible

In [11]:
digest_bytes = bytes.fromhex(digestMultiHash_hex)

# Construir el multihash: <multihash><length><digest>

hex_length = int(len_multihash, 16)
hex_codMultihasg =  int(cod_multihash, 16)

multihash_bytes = bytes([hex_codMultihasg, hex_length]) + digest_bytes

# Codificar en base32 (sin padding, en minúsculas, con prefijo 'b')
multihash_b32 = "b" + base64.b32encode(multihash_bytes).decode("utf-8").lower().rstrip("=")
print(f"El multibase del valor digest es: {multihash_b32}")

El multibase del valor digest es: bciqlsw6nnkq2nz4zxyb4jpqqat4b4lndjqgojrlnlblx4six7wtqssa


Y con el CID, el nodo de IPFS consigue:

- Saber cómo decodificar el bloque de datos (gracias al multicodec).
- Identificar el algoritmo hash utilizado y la longitud esperada (por el multihash).
- Validar la integridad del contenido, comprobando que el hash calculado sobre los datos coincide con el digest incluido en el CID.

Esto permite garantizar que los datos recuperados no han sido alterados y corresponden exactamente al contenido solicitado.

---

**Ejemplo v0**.

El ejemplo que se muestra se basa en CID Inspector, por lo que debes acceder a:
<https://cid.ipfs.tech/#QmYwAPJzv5CZsnAztbCXDpM7n6Vx4Bqs5T6zyd5oMv5E9g>


Igualmente, aquí se intenta aclarar cada concepto paso a paso para v0.

> Por favor, ejecuta una a una cada celda de códgo para que no falle.

---

In [12]:
cid = "QmYwAPJzv5CZsnAztbCXDpM7n6Vx4Bqs5T6zyd5oMv5E9g"

print(f"El CID {cid}")

El CID QmYwAPJzv5CZsnAztbCXDpM7n6Vx4Bqs5T6zyd5oMv5E9g


El valor 'Qm' es implicito indicnado para v0 ques base58btc, por lo tanto, no se intenta identificar el multicode.

Igualmente, no hay multihash, es siempre sha256.

Pasamos de base58BTC a hexadecimal

In [13]:
import base58

cid_bytes = base58.b58decode(cid)
cid_hex = cid_bytes.hex()
print(f"En hexadecimal el valor es 0x{cid_hex}")

En hexadecimal el valor es 0x12209d6c2be50f70695347c6dc417a3bef89d7bc95b7f08ed5ccd87106a4ed7563bb


Ahora extraemos el multihash: https://github.com/multiformats/multihash

Está compuesto por el código, longuitud y digest (es decir, el valor tras aplicar la función hash)

El código de multihash, posición 1 y 2.

Indica el algoritmo usado para genear el hash, que será en realiada siempre SHA-256

In [14]:
# Para mostrar el nombre del multicodec, usamos la libreria multiformats
from multiformats import multicodec

cod_multihash = cid_hex[0:2]
cod_multihash_int = int(cod_multihash, 16) # pasarlo de hexadecimal a entero
print(cod_multihash_int)
mh = multicodec.get(code=cod_multihash_int)
print(f"Código: 0x{cod_multihash}")
print(f"Nombre: {mh.name}")

18
Código: 0x12
Nombre: sha2-256


La longuitud del multihash, posición 3 y 4

In [16]:
print(cid_hex)
len_multihash = cid_hex[2:4]
print(f"Longuitud 0x{len_multihash} o {int(len_multihash,16)} bits")

12209d6c2be50f70695347c6dc417a3bef89d7bc95b7f08ed5ccd87106a4ed7563bb
Longuitud 0x20 o 32 bits


El valor digest, es decir, el propio hash aplicando la función concreta que se especifica en multicodec.

Posición desde 5 al final.

In [17]:
digestMultiHash_hex = cid_hex[4:]
print(f"Digest: {digestMultiHash_hex.upper()}")

Digest: 9D6C2BE50F70695347C6DC417A3BEF89D7BC95B7F08ED5CCD87106A4ED7563BB


Ahora multihash, se puede representa como multibase para que sea más legible

In [18]:
import base58

digest_bytes = bytes.fromhex(digestMultiHash_hex)

# Construir el multihash: <multihash><length><digest>

hex_length = int(len_multihash, 16)
hex_codMultihasg =  int(cod_multihash, 16)

multihash_bytes = bytes([hex_codMultihasg, hex_length]) + digest_bytes

# Codificar en base32 (sin padding, en minúsculas, con prefijo 'b')
multihash_b58btc = base58.b58encode(multihash_bytes).decode("utf-8")

# Agregar el prefijo 'z' para indicar que es base58btc
multibase_b58btc = "z" + multihash_b58btc
print(f"El multibase del valor digest es: {multibase_b58btc}")

El multibase del valor digest es: zQmYwAPJzv5CZsnAztbCXDpM7n6Vx4Bqs5T6zyd5oMv5E9g


**Ejemplo pasando de v0 a v1**.

Cuando un nodo gateway de IPFS recibe una petición con un CID v0, por ejemplo, en una URL como <https://gateway-ipfs.tld/ipfs/QmYwAPJzv5CZsnAztbCXDpM7n6Vx4Bqs5T6zyd5oMv5E9g>, internamente lo convierte a un CID v1 codificado en Base32 y, en muchos casos, responde redirigiendo la solicitud a una nueva URL con ese CID v1, ya que este formato es más adecuado para su uso en subdominios, compatible con DNS, insensible a mayúsculas para entornos web.

Vamos a ver este ejemplo de pasar de v0 a v1:

In [19]:
import base58
import base64

# CID v0
cid_v0 = "QmYwAPJzv5CZsnAztbCXDpM7n6Vx4Bqs5T6zyd5oMv5E9g"
print(f"CID v0: {cid_v0}")

# Decodificamos el CID v0 de base58btc a bytes
cid_bytes = base58.b58decode(cid_v0)

# Para v1, incluimos el prefijo de versión (0x01) y el multicodec 'dag-pb' (0x70)
# El multicodec 'dag-pb' (0x70) se utiliza porque los CIDs v0 siempre apuntan a bloques de datos en formato 'dag-pb' en IPFS.
cid_v1_bytes = b'\x01\x70' + cid_bytes

# Codificamos en base32 (minúsculas, sin padding) y anteponemos 'b' que es el multibase para CID v1
cid_v1_b32 = "b" + base64.b32encode(cid_v1_bytes).decode("utf-8").lower().rstrip("=")

print(f"CID v1: {cid_v1_b32}")

CID v0: QmYwAPJzv5CZsnAztbCXDpM7n6Vx4Bqs5T6zyd5oMv5E9g
CID v1: bafybeie5nqv6kd3qnfjuprw4if5dx34j266jln7qr3k4zwdra2so25ldxm


> Ahora si vuelves a https://cid.ipfs.tech/#QmYwAPJzv5CZsnAztbCXDpM7n6Vx4Bqs5T6zyd5oMv5E9g ya verás como se genera el CIDV1 (BASE32)